# Import all dependencies

In [1]:
# This is important!
import os
os.environ['TF_ENABLE_CONTROL_FLOW_V2'] = '1'

import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.lite.experimental.examples.lstm.rnn import bidirectional_dynamic_rnn
from tensorflow.keras.callbacks import LambdaCallback
import tensorflow.keras.backend as K

import json
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
import string

import keras
from keras.models import Model,Sequential, Input, load_model
from keras.layers.merge import add
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Lambda
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

Using TensorFlow backend.


# Reading and tokenizing of data

In [2]:
vocab=[]
entities=[]

with open('result_usbank_1.json') as json_file:
    dataset = json.load(json_file)
    for data in dataset:
    	sent=data['sentence']
    	tags=data['tags']
    	entities.extend(tags)
    	for word in sent:
    		vocab.append(word)
json_file.close()

vocab=set(vocab)
t=Tokenizer(num_words=len(vocab))
t.fit_on_texts(vocab)
tokens=t.word_index

num_words=len(tokens)+1

reverse_tokens = dict([(value, key) for key, value in tokens.items()])

entities=set(entities)
entities=sorted(entities)
entities=list(entities)
label_encoder=LabelEncoder()
int_encoded=label_encoder.fit_transform(entities)
label_cat_encoded=to_categorical(int_encoded,num_classes=len(entities))

master={}
for i in range(len(entities)):
	#master[entities[i]]=int_encoded[i]
	master[entities[i]]=i
print(master)

reverse_master = dict([(value, key) for key, value in master.items()])

{'amount': 0, 'bill_type': 1, 'other': 2, 'payment_day': 3, 'payment_month': 4, 'payment_type': 5}


# Pre-processing functions

In [3]:
def preparation(sent,tags,tokens,master):
    dummy_x=[]
    dummy_y=[]
    for word,tag in zip(sent,tags):
        dummy_x.append(tokens[word.lower()])
        dummy_y.append(master[tag])
    return dummy_x,dummy_y

def pre_process(sent,tokens):
    zero_tok=0.0
    sent=sent.split(" ")
    dummy_in=[]
    for word in sent:
        if word.lower() not in tokens:
            dummy_in.append(zero_tok)
        else:
            dummy_in.append(tokens[word.lower()])
    return dummy_in

# Preparation of data

In [4]:
max_length=20
X=[]
Y=[]
with open('result_usbank_1.json') as json_file:
    dataset = json.load(json_file)
    for data in dataset:
        sent=data['sentence']
        tags=data['tags']
        tokenized_x,encoded_y=preparation(sent,tags,tokens,master)
        X.append(tokenized_x)
        Y.append(encoded_y)

X=pad_sequences(X,maxlen=max_length,padding='post',dtype=float)
y=pad_sequences(Y,maxlen=max_length,padding='post')
y = y.reshape(y.shape[0],y.shape[1])
y = to_categorical(y,num_classes=len(master))
X_train, X_test, y_train, y_test=train_test_split(X,Y,test_size=0.2)
len(X)

451

# Build Bi-directional LSTM layer

In [ ]:
def build_LSTM_layer(num_layers):
    lstm_layers=[]
    for i in range(num_layers):
        lstm_layers.append(tf.lite.experimental.nn.TFLiteLSTMCell(num_units=64,name='rnn{}'.format(i),forget_bias=1.0))
    final_lstm_layer=tf.keras.layers.StackedRNNCells(lstm_layers)
    return final_lstm_layer

def build_bidirectional(inputs,num_layers,use_dynamic_rnn=True):
    lstm_inputs=transposed_inp=tf.transpose(inputs,[0,1,2])
    outputs,output_states=bidirectional_dynamic_rnn(build_LSTM_layer(num_layers),build_LSTM_layer(num_layers),lstm_inputs,dtype="float32",time_major=True)
    fw_lstm_output,bw_lstm_output=outputs
    final_out=tf.concat([fw_lstm_output,bw_lstm_output],axis=2)
    #final_out=tf.unstack(final_out,axis=0)
    resultant_out=final_out
    return resultant_out

# Build uni-directional LSTM layer

In [ ]:
def buildLstmLayer(inputs, num_layers, num_units):
  """Build the lstm layer.

  Args:
    inputs: The input data.
    num_layers: How many LSTM layers do we want.
    num_units: The unmber of hidden units in the LSTM cell.
    
  """
  lstm_cells = []
  for i in range(num_layers):
    lstm_cells.append(
        tf.lite.experimental.nn.TFLiteLSTMCell(
            num_units, forget_bias=0, name='rnn{}'.format(i)))
  lstm_layers = tf.keras.layers.StackedRNNCells(lstm_cells)
  # Assume the input is sized as [batch, time, input_size], then we're going
  # to transpose to be time-majored.
  transposed_inputs = tf.transpose(
      inputs, perm=[0,1, 2])
  outputs, _ = tf.lite.experimental.nn.dynamic_rnn(
      lstm_layers,
      transposed_inputs,
      dtype='float32',
      time_major=True)
  #unstacked_outputs = tf.unstack(outputs, axis=0)
  #return unstacked_outputs[-1]
  return outputs

In [5]:
# # model_new = tf.keras.models.Sequential([
# #   tf.keras.Input(shape=(X.shape[1],), name='input'),
# #   tf.keras.layers.Embedding(input_dim=len(vocab)+1,output_dim=32,input_length=X.shape[1],trainable=True),
# #   tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32,return_sequences=True)),
# #   tf.keras.layers.Dense(len(entities),activation=tf.nn.softmax,name='output')
# # ])

input_layer=Input(shape=(X.shape[1],))
model=Embedding(input_dim=len(vocab)+1,output_dim=32,input_length=X.shape[1])(input_layer)
model = Bidirectional(LSTM(units = 32,return_sequences=True, recurrent_dropout=0.2)) (model)
output_layer= Dense(len(master), activation="softmax")(model)
model_new = Model(input_layer,output_layer)

model_new.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model_new.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 20)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 20, 32)            5408      
_________________________________________________________________
bidirectional_1 (Bidirection (None, 20, 64)            16640     
_________________________________________________________________
dense_1 (Dense)              (None, 20, 6)             390       
Total params: 22,438
Trainable params: 22,438
Non-trainable params: 0
_________________________________________________________________


# Model architecture for bi-directional

In [ ]:
tf.reset_default_graph()
input_l = tf.keras.Input(shape=(X.shape[1],), name='input')
emb_l = tf.keras.layers.Embedding(input_dim=len(vocab)+1,output_dim=32,input_length=X.shape[1])(input_l)
lmbda_l = tf.keras.layers.Lambda(build_bidirectional, arguments={'num_layers' : 2, 'use_dynamic_rnn': True})(emb_l)
output_l = tf.keras.layers.Dense(len(entities),activation=tf.nn.softmax,name='output')(lmbda_l) 
model_tf = tf.keras.Model(input_l,output_l)
model_tf.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model_tf.summary()

# Model architecture for uni-directional 

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Input(shape=(X.shape[1],), name='input'),
  tf.keras.layers.Embedding(input_dim=len(vocab)+1,output_dim=32,input_length=X.shape[1],trainable=True),
  tf.keras.layers.Lambda(buildLstmLayer, arguments={'num_layers' : 2, 'num_units' : 64}),
  tf.keras.layers.Dense(len(entities),activation=tf.nn.softmax,name='output')
])
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

# Model training

In [6]:
def batchOutput(batch, logs):
    print("Finished batch: " + str(batch))

batchLogCallback = LambdaCallback(on_batch_end=batchOutput)
train_x,test_x,train_y,test_y = train_test_split(X,y,test_size=0.2)

history = model_new.fit(train_x,train_y,epochs=100,batch_size=16)
model_new.save('custom_ner_usbank.h5')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/100
360/360 [==============================] - 1s 3ms/step - loss: 1.6469 - accuracy: 0.7007
Epoch 2/100
360/360 [==============================] - 0s 967us/step - loss: 0.9814 - accuracy: 0.7499
Epoch 3/100
360/360 [==============================] - 0s 1ms/step - loss: 0.6776 - accuracy: 0.8132
Epoch 4/100
360/360 [==============================] - 0s 885us/step - loss: 0.5885 - accuracy: 0.8383
Epoch 5/100
360/360 [==============================] - 0s 863us/step - loss: 0.5460 - accuracy: 0.8419
Epoch 6/100
360/360 [==============================] - 0s 890us/step - loss: 0.5020 - accuracy: 0.8456
Epoch 7/100
360/360 [==============================] - 0s 896us/step - loss: 0.4564 - accuracy: 0.8478
Epoch 8/100
360/360 [==============================] - 0s 863us/step - loss: 0.4096 - accuracy: 0.8561
Epoch 9/100
360/360 [==============================] - 0s 975us/step - loss: 0.3559 -

360/360 [==============================] - 0s 888us/step - loss: 0.0055 - accuracy: 0.9990
Epoch 75/100
360/360 [==============================] - 0s 862us/step - loss: 0.0051 - accuracy: 0.9990
Epoch 76/100
360/360 [==============================] - 0s 839us/step - loss: 0.0054 - accuracy: 0.9989
Epoch 77/100
360/360 [==============================] - 0s 865us/step - loss: 0.0052 - accuracy: 0.9990
Epoch 78/100
360/360 [==============================] - 0s 852us/step - loss: 0.0053 - accuracy: 0.9990
Epoch 79/100
360/360 [==============================] - 0s 1ms/step - loss: 0.0052 - accuracy: 0.9990
Epoch 80/100
360/360 [==============================] - 0s 1ms/step - loss: 0.0052 - accuracy: 0.9990
Epoch 81/100
360/360 [==============================] - 0s 846us/step - loss: 0.0052 - accuracy: 0.9990
Epoch 82/100
360/360 [==============================] - 0s 896us/step - loss: 0.0049 - accuracy: 0.9990
Epoch 83/100
360/360 [==============================] - 0s 1ms/step - loss: 0.004

In [7]:
print(tokens['credit'])

82


In [9]:
temp='pay my barclays bill using credit card'
length=temp.split(" ")
tok_sent=pre_process(temp,tokens)
for i in range(max_length):
    if len(tok_sent)<max_length:
        tok_sent.append(0.0)
tok_sent=np.expand_dims(tok_sent,axis=0)
print(tok_sent)
result=model_new.predict(tok_sent)
for i in range(len(length)):
    ind=np.argmax(result[0][i])
    print(length[i],"----->",reverse_master[ind])

[[  5. 150. 107.  76. 122.  82. 124.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.]]
pay -----> other
my -----> other
barclays -----> bill_type
bill -----> other
using -----> other
credit -----> payment_type
card -----> other


# Converting to tflite

In [ ]:
sess = tf.keras.backend.get_session()
input_tensor = sess.graph.get_tensor_by_name('input:0')
output_tensor = sess.graph.get_tensor_by_name('output/Softmax:0')
converter = tf.lite.TFLiteConverter.from_session(
    sess, [input_tensor], [output_tensor])
tflite = converter.convert()
open("custom_ner_sequential_updated_1.tflite","wb").write(tflite)
print('Model converted successfully!')

# Inferencing with the tflite model

In [ ]:
input_text="when did i take 10 mg of olumiant in last week"
words=input_text.split(" ")
text=pre_process(input_text,tokens)
final=[]
for i in range(max_length):
    if i>=len(text):
        final.append(0)
    else:
        final.append(text[i])
ip = np.array(final,dtype="float32")
ip=[ip]
ip

In [ ]:
interpreter = tf.lite.Interpreter(model_content=tflite)
try:
    interpreter.allocate_tensors()
except ValueError:
    assert False
    
input_index = (interpreter.get_input_details()[0]['index'])
interpreter.set_tensor(input_index,ip)
interpreter.invoke()
output_index = (interpreter.get_output_details()[0]['index'])
result = interpreter.get_tensor(output_index)
print(result)

for i in range(len(result[0])):
    if i>=len(words):
        pass
    else:
        tag=reverse_master[np.argmax(result[0][i])]
        print(words[i],tag)
    

# Export the vocab

In [ ]:
with open('vocab_ner_usbank.json','w') as f:
    json.dump(tokens,f)